In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

In [2]:
df = pd.read_csv('./data/updated_data.csv')
df.head()

,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price
0,95145,60,0,1,60,34773,1,4,2000,0,1,729,4496,277,0,6,9519958.0
1,92661,45,1,1,62,45457,4,8,2020,1,1,7473,8953,245,1,9,9276448.1
2,97184,99,0,0,59,15113,1,1,2000,0,1,6424,8522,256,1,9,9725732.2
3,61752,100,0,0,57,64245,8,4,2018,1,0,7151,2786,863,0,7,6181908.8
4,30300,36,0,0,35,19069,8,4,1998,1,0,8181,1826,589,0,0,3033117.8


In [3]:
print(df.shape)

(20726, 17)


In [4]:
X = df.drop(labels=['price'],axis=1)
Y = df[['price']]

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Numerical pipeline
num_pipeline = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]
)

# Apply pipeline to the entire DataFrame of features
X_transformed = num_pipeline.fit_transform(X)


In [6]:
print(X_transformed.shape)

(20726, 16)


In [8]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X_transformed,Y,test_size=0.2,random_state=42)
xtrain.shape, ytrain.shape, xtest.shape

((16580, 16), (16580, 1), (4146, 16))

In [9]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [16]:
print(type(xtrain))  # Output: <class 'pandas.DataFrame'> or <class 'numpy.ndarray'>

# Check the data type of y (target)
print(type(ytrain.values.flatten()))

print(xtrain.shape)  # Output: (n_samples, n_features)

# Check the shape of y (target)
print((ytrain.values.flatten()).shape)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(16580, 16)
(16580,)


In [17]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(xtrain, ytrain.values.flatten()) # Train model

    # Make predictions
    y_train_pred = model.predict(xtrain)
    y_test_pred = model.predict(xtest)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(ytrain, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(ytest, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')

    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 185459.1833
- Mean Absolute Error: 13464.7883
- R2 Score: 0.9959
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 223900.4935
- Mean Absolute Error: 15389.3299
- R2 Score: 0.9939


Lasso
Model performance for Training set
- Root Mean Squared Error: 185459.1833
- Mean Absolute Error: 13463.3912
- R2 Score: 0.9959
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 223900.4043
- Mean Absolute Error: 15387.9438
- R2 Score: 0.9939


Ridge
Model performance for Training set
- Root Mean Squared Error: 185459.2674
- Mean Absolute Error: 13535.7014
- R2 Score: 0.9959
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 223897.1942
- Mean Absolute Error: 15462.5778
- R2 Score: 0.9939


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 954169.8890
- Mean Absolu